# Implementações dos problemas de PLI - A2
### Disciplina: Programação Linear Inteira
### Integrantes: Daniel Jacob, Juliana Carrica, Wendell Oliveira
### Professor: Vincent Gerard Y. Guigues

In [53]:
import Pkg; Pkg.add("MathOptInterface")
import Pkg; Pkg.add("Plots")
import Pkg; Pkg.add("DataFrames")

   Resolving package versions...
  No Changes to `C:\Users\Daniel\.julia\environments\v1.10\Project.toml`
  No Changes to `C:\Users\Daniel\.julia\environments\v1.10\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\Daniel\.julia\environments\v1.10\Project.toml`
  No Changes to `C:\Users\Daniel\.julia\environments\v1.10\Manifest.toml`
   Resolving package versions...
   Installed InvertedIndices ───────────── v1.3.0
   Installed DataValueInterfaces ───────── v1.0.0
   Installed SentinelArrays ────────────── v1.4.1
   Installed TableTraits ───────────────── v1.0.1
   Installed PooledArrays ──────────────── v1.4.3
   Installed InlineStrings ─────────────── v1.4.0
   Installed Crayons ───────────────────── v4.1.1
   Installed DataFrames ────────────────── v1.6.1
   Installed IteratorInterfaceExtensions ─ v1.0.0
   Installed Tables ────────────────────── v1.11.1
   Installed StringManipulation ────────── v0.3.4
   Installed PrettyTables ──────────────── v2.3.1
    Up

In [54]:
#Bibliotecas 
using JuMP
using GLPK
using Plots
using DataFrames

# Problema 1.1

In [14]:
# Definindo o modelo
model = Model(GLPK.Optimizer)

# Variáveis de decisão
@variable(model, q[1:2, 1:3] >= 0, Int)

# Função objetivo
c = [1 2 1.5; 2 3.5 2]  # Custos de transporte organizados por usina
@objective(model, Min, sum(q[i, j] * c[i, j] for i in 1:2, j in 1:3))

# Restrições de capacidade das usinas
@constraint(model, sum(q[1, j] for j in 1:3) <= 400)
@constraint(model, sum(q[2, j] for j in 1:3) <= 300)

# Restrições de demanda das fábricas
@constraint(model, sum(q[i,1] for i in 1:2) == 100)
@constraint(model, sum(q[i,2] for i in 1:2) == 200)
@constraint(model, sum(q[i,3] for i in 1:2) == 300)

# Resolvendo o modelo
optimize!(model)

# Imprimindo o resultado
println("Custo mínimo: ", objective_value(model))
println("Quantidades ótimas de transporte:")
for i in 1:2
    for j in 1:3
        println("q[$i, $j] = ", value(q[i, j]))
    end
end

# Imprimindo excedente no estoque
println("Excedente na usina 1: ", 400 -  value(sum(q[1, j] for j in 1:3)) )
println("Excedente na usina 2: ", 300 -  value(sum(q[2, j] for j in 1:3)) )

Custo mínimo: 1050.0
Quantidades ótimas de transporte:
q[1, 1] = 100.0
q[1, 2] = 200.0
q[1, 3] = 100.0
q[2, 1] = 0.0
q[2, 2] = 0.0
q[2, 3] = 200.0
Excedente na usina 1: 0.0
Excedente na usina 2: 100.0


# Problema 1.2

In [63]:
# Definindo o modelo
model_first = Model(GLPK.Optimizer)

# Variáveis de decisão
@variable(model_first, T[1:2] >= 0, Int)

# Função objetivo
c = [7 9]  # Lucro por kilo de produto
@objective(model_first, Max, sum(T[j] * c[j] for j in 1:2))

# Restrições de limitação de matéria-prima
@constraint(model_first, T[1] + T[2] <= 8)
@constraint(model_first, 2T[1] + 3T[2] <= 19)
@constraint(model_first, T[2] <= 4)

# Resolvendo o modelo]
optimize!(model_first)

# Imprimindo o resultado
println("Lucro máximo: ", objective_value(model_first))
println("Quantidades ótimas de produção:")
for i in 1:2
    println("T[$i] = ", value(T[i]))
end

#Exibindo o excedente de matéria-prima
println("Excedente Nitrato: ", 8 - ( value(T[1]) + value(T[2]) ))
println("Excedente Sal de Potássio: ", 19 - ( 2value(T[1]) + 3value(T[2])))
println("Excedente Fosfato: ", 4 -  (value(T[2]) ))




Lucro máximo: 62.0
Quantidades ótimas de produção:
T[1] = 5.0
T[2] = 3.0
Excedente Nitrato: 0.0
Excedente Sal de Potássio: 0.0
Excedente Fosfato: 1.0


In [64]:
# Definindo o modelo
model_sec = Model(GLPK.Optimizer)

# Variáveis de decisão
@variable(model_sec, P[1:3] >= 0, Int)

# Função objetivo
c = [8, 19, 4]  # Quantidade de nutrientes por quilo de produto disponível
@objective(model_sec, Min, sum(P[i] * c[i] for i in 1:3))

# Restrições de lucro por quilo de produto
@constraint(model_sec, P[1] + 2* P[2] >= 700)
@constraint(model_sec, P[1] +3P[2] + P[3] >= 900)

# Resolvendo o modelo
optimize!(model_sec)

# Imprimindo o resultado
println("Minimização do preço à granel: ", objective_value(model_sec))
println("Preço por granel:")
for i in 1:3
    println("P[$i] = ", value(P[i]))
end


Minimização do preço à granel: 6200.0
Preço por granel:
P[1] = 300.0
P[2] = 200.0
P[3] = 0.0


# Problema 1.3

In [66]:
# Definindo o modelo
model = Model(GLPK.Optimizer)

# Variáveis de decisão
@variable(model, q[1:2 , 1:3] >= 0, Int)

# Função objetivo
c = [38 27 48 ; 37 58 45]  # Custo de transporte por rota
@objective(model, Max, sum(q[i,j] * c[i,j] for i in 1:2, j in 1:3))

# Restrições de limitação de matéria-prima
@constraint(model, sum(q[1,j] for j in 1:3) <= 6)
@constraint(model, sum(q[2,j] for j in 1:3) <= 6)

# Restrições de demanda das fábricas
@constraint(model, sum(q[i,1] for i in 1:2) == 5)
@constraint(model, sum(q[i,2] for i in 1:2) == 4)
@constraint(model, sum(q[i,3] for i in 1:2) == 3)

# Resolvendo o modelo
optimize!(model)

# Imprimindo o resultado
println("Custo mínimo de distribuição: ", objective_value(model))
println("Quantidades ótimas de distribuição:")
for i in 1:2
    for j in 1:3
        println("q[$i, $j] = ", value(q[i, j]))
    end
end

#Exibindo as sobras de cada usina
println("Sobra na usina 1: ", 6 -  value(sum(q[1, j] for j in 1:3)) )
println("Sobra na usina 2: ", 6 -  value(sum(q[2, j] for j in 1:3)) )



Custo mínimo de distribuição: 564.0
Quantidades ótimas de distribuição:
q[1, 1] = 3.0
q[1, 2] = 0.0
q[1, 3] = 3.0
q[2, 1] = 2.0
q[2, 2] = 4.0
q[2, 3] = 0.0
Sobra na usina 1: 0.0
Sobra na usina 2: 0.0


# Problema 1.4

In [68]:
# Definindo o modelo
model = Model(GLPK.Optimizer)

# Variáveis de decisão
@variable(model, S >= 0, Int)
@variable(model, C >= 0, Int)

# Função objetivo
@objective(model, Max, 5S + 4C)

# Restrições de limitação de matéria-prima
@constraint(model, 2S+C <= 78)

# Restrições de horas trabalhadas
@constraint(model, 10S+12C == 600)

# Resolvendo o modelo
optimize!(model)

# Imprimindo o resultado
println("Lucro máximo: ", objective_value(model))
println("Quantidades ótimas de produção:")
println("S = ", value(S))
println("C = ", value(C))
#Horas trabalhadas 
println("Horas trabalhadas em sapatos: ", 10*value(S) )
println("Horas trabalhadas em cintos: ", 12*value(C) )


Lucro máximo: 240.0
Quantidades ótimas de produção:
S = 24.0
C = 30.0
Horas trabalhadas em sapatos: 240.0
Horas trabalhadas em cintos: 360.0


# Problema 1.5

In [70]:
# Definindo o modelo
model = Model(GLPK.Optimizer)

# Variáveis de decisão
@variable(model, QA >= 0, Int)
@variable(model, QB >= 0, Int)

# Função objetivo
@objective(model, Max, 20QA + 30QB)

# Restrições de limitação de matéria-prima
@constraint(model, (1/4)*QA <= 500)
@constraint(model, (1/4)*QA + (1/2)*QB<= 200)
@constraint(model, (1/2)*QA + (1/2)*QB<= 200)

# Resolvendo o modelo
optimize!(model)

# Imprimindo o resultado
println("Lucro máximo: ", objective_value(model))
println("Quantidades ótimas de produção:")
println("QA = ", value(QA))
println("QB = ", value(QB))

#Exibindo o excedente de matéria-prima
println("Excedente de A: ", 500 -  (1/4)*value(QA) )
println("Excedente de B: ", 200 -  (1/4)*value(QA) - (1/2)*value(QB) )
println("Excedente de C: ", 200 -  (1/2)*value(QA) - (1/2)*value(QB) )

Lucro máximo: 12000.0
Quantidades ótimas de produção:
QA = 0.0
QB = 400.0
Excedente de A: 500.0
Excedente de B: 0.0
Excedente de C: 0.0


# Problema 1.6 - a implementação diz que não tem resultado ótimo


In [89]:
# Definindo o modelo
model = Model(GLPK.Optimizer)

# Variáveis de decisão
@variable(model, q[1:4, 1:3], Int)

# Função objetivo
c = [5.5, 4.5, 3.5]  # Venda dos diesel 1,2,3
d = [3, 6, 4, 1] # Custo dos petróleos 1,2,3,4
@objective(model, Max, sum((q[i,j] * c[j]) for i in 1:4, j in 1:3) - sum((q[i,j] * d[i]) for i in 1:4, j in 1:3))

# Restrições de composição
# Diesel A
@constraint(model, (0.3) * sum(q[i,1] for i in 1:4) >= q[1,1])
@constraint(model, (0.4) * sum(q[i,1] for i in 1:4) <= q[2,1])
@constraint(model, (0.5) * sum(q[i,1] for i in 1:4) >= q[3,1])
# Diesel B
@constraint(model, (0.5) * sum(q[i,2] for i in 1:4) >= q[1,2])
@constraint(model, (0.1) * sum(q[i,2] for i in 1:4) >= q[2,2])
# Diesel C
@constraint(model, (0.7) * sum(q[i,3] for i in 1:4) >= q[1,3])

# Restrições de matéria-prima
@constraint(model, sum(q[1,j] for j in 1:3) <= 3000)
@constraint(model, sum(q[2,j] for j in 1:3) <= 2000)
@constraint(model, sum(q[3,j] for j in 1:3) <= 4000)
@constraint(model, sum(q[4,j] for j in 1:3) <= 1000)

# Resolvendo o modelo
optimize!(model)

# Imprimindo o resultado
println("Lucro máximo: ", objective_value(model))

# Quantidades ótimas de produção de cada diesel
for j in 1:3
    println("Quantidade ótima de diesel tipo $j: ", sum(value(q[i,j]) for i in 1:4))
end

# Excedentes de matéria-prima
println("Excedente de matéria-prima:")
println("Excedente de 1: ", 3000 - sum(value(q[1,j]) for j in 1:3))
println("Excedente de 2: ", 2000 - sum(value(q[2,j]) for j in 1:3))
println("Excedente de 3: ", 4000 - sum(value(q[3,j]) for j in 1:3))
println("Excedente de 4: ", 1000 - sum(value(q[4,j]) for j in 1:3))


MathOptInterface.ResultIndexBoundsError{MathOptInterface.ObjectiveValue}: Result index of attribute MathOptInterface.ObjectiveValue(1) out of bounds. There are currently 0 solution(s) in the model.

# Problema 2.1 - Gestão da produção de eletricidade

# Problema 2.2 - Gestão de carreiras

In [10]:
# Definindo o modelo
model = Model(GLPK.Optimizer)

# Variáveis de decisão
@variable(model, q[1:5] >= 0)  

# Função objetivo
c = [0.05; 0.065; 0.057; 0.052; 0.1] 
@objective(model, Max, sum(q[i] * c[i] for i in 1:5)) 
# Restrições de limitação de investimento
@constraint(model, sum(q[i] for i in 1:5) == 100000)

# Resolvendo o modelo
optimize!(model)

# Imprimindo o resultado
println("Retorno: ", objective_value(model) + 100000)  
println("Lucro obtido: ", objective_value(model)) 
println("Investimento em cada ativo:")
for i in 1:5
    println("q[$i] = ", value(q[i]))
end



Retorno: 110000.0
Lucro obtido: 10000.0
Investimento em cada ativo:
q[1] = 0.0
q[2] = 0.0
q[3] = 0.0
q[4] = 0.0
q[5] = 100000.0


# Problema 2.3 - Planificação da expansão de produção

In [111]:
# Definindo o modelo
model = Model(GLPK.Optimizer)

# custo de unidades não vendidas
p = [400 400 400]
# custo de horas extras
c = [2.5 3.75 5 3]
#tempo de revisão de cada máquina por hora trabalhada
t = [0.08 0.04 0.03 0.01]
#horas disponíveis de cada maquina
hd = [500 500 500 500]
#tempo máximo de trabalho de cada máquina
u = [2000 2000 3000 3000]
# u = [1000 1000 1000 1000]
# Taxa de produção de i em j
a = [0.6 0.6 0.9 0.8; 0.1 0.9 0.6 0.8; 0.05 0.2 0.5 0.8]
#Custo por hora de produção de i em j
q = [2.6 3.4 3.4 2.5; 1.5 2.4 2 3.6; 4 3.8 3.5 3.2]
#demanda numa semana
d = [1800 600 3000]

# Variáveis de decisão
# h[i] = horas convencionais de trabalho em cada máquina
@variable(model, hc[1:4] >= 0)  
# e[j] = horas extras de trabalho em cada máquina
@variable(model, e[1:4] >= 0)
# tempo H[i,j] de confecção de i em j
@variable(model, H[1:3, 1:4] >= 0)

# Função objetivo
@objective(model, Min, sum(q[i,j] * H[i,j] for i in 1:3, j in 1:4) + 
                sum(c[j] * e[j] for j in 1:4) +
                sum(p[i]*(d[i] - sum(a[i,j]*H[i,j] for j in 1:4)) for i in 1:3))
             
# Restrições de horas por máquina
@constraint(model,[j in 1:4], hc[j] + e[j]  <= u[j])
# Restrições de horas de revisão
@constraint(model,sum(t[j]*(e[j] + hc[j]) for j in 1:4)  <= 100 )
# Limitação de horas contratadas
@constraint(model,[j in 1:4],hc[j] <= hd[j] )
# Restrições de demanda
@constraint(model,[i in 1:3], sum(a[i,j]*H[i,j] for j in 1:4) <= d[i] )
# Restrições de soma de horas por máquina
@constraint(model,[j in 1:4], sum(H[i,j] for i in 1:3) == hc[j] + e[j] )
# Resolvendo o modelo
optimize!(model)

# Imprimindo o resultado
println("Custo total: ", round(objective_value(model)))

Custo total: 422000.0


In [113]:
# Criando uma matriz para armazenar os resultados
results = Matrix{Any}(undef, 7, 8)

#1x1 -  4x8
for j in 1:4
    results[j, 1] = "Máquina $j"
    results[j, 2] = round(value(e[j]))
    results[j, 3] = round(value(hc[j]))
    results[j, 4] = round(value(hc[j]+ e[j]))
    results[j, 5] = round(value((e[j] + hc[j])*t[j]))
    for i in 1:3
        results[j, 5 + i] = round(value(H[i,j]*a[i,j]))
    end
end

results[5, 5] = "Produzidos"
results[6, 5] = "Não vendidos"
results[7, 5] = "Demanda"
for i in 1:3
    results[5, 5+i] = round(sum(value(H[i,j]*a[i,j]) for j in 1:4))
    results[6, 5+i] = round(value(d[i] - sum(value(H[i,j]*a[i,j]) for j in 1:4)))
    results[7, 5+i] = d[i]
end
for i in 5:7
    for j in 1:4
        results[i,j] = ""
    end
end

# Criando um DataFrame
df = DataFrame(results, ["Máquina", "Extra", "Disponível", "Total", "Revisão", "Produto 1", "Produto 2", "Produto 3"])
#definindo titulo das colunas do DataFrame

# Exibindo o DataFrame
# println(df)
show(df, hlines = 5:5)


7×8 DataFrame
 Row │ Máquina    Extra   Disponível  Total   Revisão       Produto 1  Produto ⋯
     │ Any        Any     Any         Any     Any           Any        Any     ⋯
   1 │ Máquina 1  0.0     0.0         0.0     0.0           0.0        0.0     ⋯
   2 │ Máquina 2  0.0     250.0       250.0   10.0          0.0        225.0
   3 │ Máquina 3  1500.0  500.0       2000.0  60.0          1800.0     0.0
   4 │ Máquina 4  2500.0  500.0       3000.0  30.0          0.0        0.0
─────┼──────────────────────────────────────────────────────────────────────────
   5 │                                        Produzidos    1800.0     225.0   ⋯
   6 │                                        Não vendidos  0.0        375.0
   7 │                                        Demanda       1800       600
                                                               2 columns omitted

# Problema 2.4 - Gestão de contratos com opção de cancelamento

# Problema 2.5 - Problema da mochila

In [16]:
# Definindo o modelo
model = Model(GLPK.Optimizer)

# Variáveis de decisão
@variable(model, x[1:25], Bin)  

# Função objetivo
# Lista de objetos úteis para uma viagem às montanhas da Cordilheira dos Andes
objetos = [
    "Barraca resistente ao frio", "Saco para baixas temperaturas",
    "Papel Higiênico", "Panela leve para cozinhar", "Canivete suíço",
    "Cordas de escalada", "Capacete de escalada", "Mosquetões",
    "Roupas térmicas", "Luvas resistentes ao frio", "Gorro de lã",
    "Óculos de sol com proteção UV", "Botas de trekking impermeáveis",
    "Mapa da região", "GPS portátil", "Cantil de água", "Comida em pó",
    "Protetor solar de alta proteção", "Kit de primeiros socorros",
    "Lanterna de cabeça", "Bússola", "Kit escalada",
    "Bolsa estanque para objetos importantes", "Mochila de trekking",
    "Garrafa térmica"
]

# Lista de valores numéricos da utilidade de cada objeto
utilidade = [
    9, 10, 8, 7, 6, 9, 9, 8, 10, 9, 8, 7, 9, 7, 8, 7, 9, 7, 10, 8, 7, 8, 7, 8, 9, 8
]

# Lista de pesos dos objetos (em kg)
pesos = [
    3, 2, 1, 0.5, 0.2, 4, 0.5, 0.3, 2, 0.3, 0.1, 0.1, 1.5, 0.1, 0.3, 0.2, 0.5, 0.1, 0.5,
    0.3, 0.2, 0.3, 0.2, 2, 0.5
]

@objective(model, Max, sum(utilidade[i] * x[i] for i in 1:25)) 

# Restrições de limite de carga
@constraint(model, sum( x[i] * pesos[i] for i in 1:25) <= 10)

# Resolvendo o modelo
optimize!(model)

# Imprimindo o resultado
println("Utilidade máxima: ", objective_value(model))  
println("Peso atingido: ", sum(pesos[i]*value(x[i]) for i in 1:25)) 
println("Objetos escolhidos:")
contador = 1
for i in 1:25
    if value(x[i]) > 0
        println(contador,".", objetos[i])
        contador+=1
    end
end


Utilidade máxima: 168.0
Peso atingido: 9.7
Objetos escolhidos:
1.Papel Higiênico
2.Panela leve para cozinhar
3.Canivete suíço
4.Capacete de escalada
5.Mosquetões
6.Roupas térmicas
7.Luvas resistentes ao frio
8.Gorro de lã
9.Óculos de sol com proteção UV
10.Botas de trekking impermeáveis
11.Mapa da região
12.GPS portátil
13.Cantil de água
14.Comida em pó
15.Protetor solar de alta proteção
16.Kit de primeiros socorros
17.Lanterna de cabeça
18.Bússola
19.Kit escalada
20.Bolsa estanque para objetos importantes
21.Garrafa térmica


# Problema 2.6 - Unit Commitment